## Carefull, very much work in progress
Currently, uses a single items sell,buy, sellvol,buyvol to predict.
This may not work
Other ideas:
Using multiple items

Uses min-max scaler, may not be working correctly.

In [ ]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [ ]:
import mlflow
import pandas as pd
import numpy as np
import matplotlib

In [ ]:
# Download source data from s3
from ml.common.folder_s3_sync import download_folder_from_s3

datapath = "../../../data/train_prophet/"


download_folder_from_s3(s3_folder_prefix='datasources/gw2/raw/1762686861', local_folder=datapath)
df = pd.read_csv(f'{datapath}Copper Ore.csv', delimiter=";")


In [ ]:
from darts import TimeSeries
#value_cols = ["buy_unit_price","sell_quantity", "sell_unit_price", "buy_quantity"]

value_cols = ["buy_unit_price"]

tsdf = df[value_cols + ['fetched_at']].copy()
# series = TimeSeries.from_dataframe(df, time_col="fetched_at")

# Convert to datetime and set as index
tsdf['fetched_at'] = pd.to_datetime(tsdf['fetched_at'])
# Localize timezone to UTC if it has timezone info, or remove it
if tsdf['fetched_at'].dt.tz is not None:
    tsdf['fetched_at'] = tsdf['fetched_at'].dt.tz_localize(None)
tsdf = tsdf.set_index('fetched_at')

# Resample to exact 5-minute intervals, forward-filling missing values
tsdf_resampled = tsdf[value_cols].resample('5min').mean().interpolate(method='linear')

# Create TimeSeries (no need for fill_missing_dates now)
series = TimeSeries.from_dataframe(tsdf_resampled, value_cols=value_cols)
#series.plot();
series.plot()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Scaling the data
from darts.dataprocessing.transformers import Scaler
scaler = Scaler(scaler=MinMaxScaler())
series_scaled = scaler.fit_transform(series)
series_scaled.plot()

In [ ]:
train, val = series_scaled.split_after(0.75)
train.plot(label="training")
val.plot(label="validation");


In [ ]:
from darts.metrics import mape, rmse, mae
from darts.models import Prophet


model = Prophet()
model.fit(train, verbose=True)

# Make predictions
forecast = model.predict(len(val))

# Inverse transform for real-world metrics
forecast_original = scaler.inverse_transform(forecast)
val_original = scaler.inverse_transform(val)

# Calculate multiple metrics
error_mape = mape(val_original, forecast_original)
error_rmse = rmse(val_original, forecast_original)
error_mae = mae(val_original, forecast_original)

print(f"MAPE: {error_mape:.2f}%")
print(f"RMSE: {error_rmse:.2f}")
print(f"MAE: {error_mae:.2f}")

# Plot results
forecast_original.plot(label="forecast")
val_original.plot(label="validation")


In [ ]:
forecast.plot(label="forcast")
val.plot(label="validation")

In [ ]:
import mlflow
import os

mlflow.set_experiment("FBProphet")
print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")

In [ ]:
from darts.metrics import mape
from darts.models import Prophet
import mlflow
import os

print(mlflow.get_tracking_uri())

best_mape = float('inf')
best_params = None

# Define Prophet hyperparameter search space
seasonality_modes = ['additive', 'multiplicative']
changepoint_prior_scales = [0.001, 0.01, 0.1, 0.5]
seasonality_prior_scales = [0.01, 0.1, 1.0, 10.0]

# Parent run for the hyperparameter search
with mlflow.start_run(run_name="FBProphet_GridSearch"):

    # Log search configuration
    mlflow.log_param("seasonality_modes", str(seasonality_modes))
    mlflow.log_param("changepoint_prior_scales", str(changepoint_prior_scales))
    mlflow.log_param("seasonality_prior_scales", str(seasonality_prior_scales))
    mlflow.log_param("total_combinations", len(seasonality_modes) * len(changepoint_prior_scales) * len(seasonality_prior_scales))

    # Grid search over hyperparameters
    for seasonality_mode in seasonality_modes:
        for changepoint_prior_scale in changepoint_prior_scales:
            for seasonality_prior_scale in seasonality_prior_scales:

                params_str = f"{seasonality_mode}_cp{changepoint_prior_scale}_sp{seasonality_prior_scale}"
                print(f"Testing Prophet: {params_str}")

                # Child run for each parameter combination
                with mlflow.start_run(run_name=f"Prophet_{params_str}", nested=True):
                    # Log parameters
                    mlflow.log_param("seasonality_mode", seasonality_mode)
                    mlflow.log_param("changepoint_prior_scale", changepoint_prior_scale)
                    mlflow.log_param("seasonality_prior_scale", seasonality_prior_scale)
                    mlflow.log_param("model_type", "Prophet")

                    try:
                        # Train model
                        model = Prophet(
                            seasonality_mode=seasonality_mode,
                            changepoint_prior_scale=changepoint_prior_scale,
                            seasonality_prior_scale=seasonality_prior_scale
                        )
                        model.fit(train)

                        # Make predictions
                        forecast = model.predict(len(val))

                        # Inverse transform for real-world metrics
                        forecast_original = scaler.inverse_transform(forecast)
                        val_original = scaler.inverse_transform(val)

                        # Calculate metrics
                        error_mape = mape(val_original, forecast_original)
                        error_rmse = rmse(val_original, forecast_original)
                        error_mae = mae(val_original, forecast_original)

                        # Log metrics
                        mlflow.log_metric("mape", error_mape)
                        mlflow.log_metric("rmse", error_rmse)
                        mlflow.log_metric("mae", error_mae)



                        # Check if this is the best model so far
                        if error_mape < best_mape:
                            best_mape = error_mape
                            best_params = {
                                'seasonality_mode': seasonality_mode,
                                'changepoint_prior_scale': changepoint_prior_scale,
                                'seasonality_prior_scale': seasonality_prior_scale
                            }
                            print(f"New best: {params_str} with MAPE: {error_mape:.2f}%")

                            # Log as best model in parent run
                            mlflow.log_metric("best_mape", error_mape)
                            mlflow.log_param("best_seasonality_mode", seasonality_mode)
                            mlflow.log_param("best_changepoint_prior_scale", changepoint_prior_scale)
                            mlflow.log_param("best_seasonality_prior_scale", seasonality_prior_scale)

                             # Save model to file and log as artifact
                            model.save("../../../models/prophet_model.pkl")
                            mlflow.log_artifact("../../../models/prophet_model.pkl")

                    except Exception as e:
                        # Log failed runs
                        mlflow.log_param("status", "failed")
                        mlflow.log_param("error", str(e))
                        print(f"Failed Prophet({params_str}): {str(e)}")

    # Log final results in parent run
    mlflow.log_metric("final_best_mape", best_mape)
    mlflow.log_param("final_best_params", str(best_params))

    print(f"\nBest model: Prophet{best_params} with MAPE: {best_mape:.2f}%")

In [ ]:
best_model = Prophet(
    seasonality_mode='additive',
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=0.01
)

best_model.fit(series_scaled)

# Forecast next 7 days
forecast_horizon = 7 * 24 * 12  # 7 days in 5-minute intervals
forecast = best_model.predict(forecast_horizon)

# Transform back to original scale
forecast_original = scaler.inverse_transform(forecast)
series_original = scaler.inverse_transform(series_scaled)

# Plot
series_original.plot(label="Historical")
forecast_original.plot(label="Forecast (7 days)")


In [ ]:
from darts.models import NaiveDrift, VARIMA
from darts.metrics import mape

def eval_model(model):

    model.fit(train)
    forecast = model.predict(len(val))
    print(f"model {model} obtains MAPE: {mape(val, forecast):.2f}%")


eval_model(NaiveDrift())
eval_model(VARIMA())
eval_model(VARIMA(p=3, d=1, q=2))